# COVID-19 — Vaccination Leaders

## Intro
This dashboard compares countries by COVID-19 vaccinations.  
On the left you see the top countries in absolute numbers,  
on the right the leaders relative to their population.

In [1]:
# --- Imports ---

import numpy as np
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# --- Load data ---
file_path = "../Data/Covid_19_dataset.csv"
df = pd.read_csv(file_path)

# Ensure datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# --- Project palette (consistent with previous notebooks) ---
project_palette = ["#0474ed","#91f0fa","#08a29e","#a2b458","#a6cabd","#326164"]

# for Plotly Express figures in Dash
palette_dash = project_palette

In [13]:
# -- Cleaning & snapshot (last day per country)

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df[~df["iso_code"].str.startswith("OWID_")].copy()
df = df.sort_values(["iso_code", "date"])

# forward-fill so the last day holds latest totals
df["total_vaccinations"] = df.groupby("iso_code")["total_vaccinations"].ffill()

# last row per country
last_rows = df.groupby("iso_code", as_index=False).tail(1)

countries = last_rows[["location", "population", "total_vaccinations"]].copy()
countries = countries[
    countries["population"].notna()
    & countries["total_vaccinations"].notna()
    & (countries["population"] > 0)
].reset_index(drop=True)

countries["vaccinations_per_capita"] = countries["total_vaccinations"] / countries["population"]


# -- Dash app — 1200×600 white box with two side-by-side bars

BOX_W, BOX_H = 1200, 600
CHART_W, CHART_H = 580, 460

app = Dash(__name__)
app.title = "COVID-19 — Vaccination Leaders"

app.layout = html.Div(
    style={"backgroundColor": "#ffffff", "padding": "12px", "fontFamily": "Arial, sans-serif"},
    children=[
        html.Div(
            style={"width": f"{BOX_W}px", "height": f"{BOX_H}px", "margin": "0 auto",
                   "display": "flex", "flexDirection": "column"},
            children=[
                html.H3("Top countries by vaccinations", style={"margin": "0 0 8px 0"}),

                html.Div([
                    html.Label("Select N (number of countries):", style={"fontWeight": 600}),
                    html.Div(
                        dcc.Slider(
                            id="n-slider", min=5, max=20, step=5, value=10,
                            marks={i: str(i) for i in [5, 10, 15, 20]},
                            tooltip={"placement": "bottom", "always_visible": True},
                        ),
                        style={"width": "420px"}
                    ),
                ], style={"marginBottom": "8px"}),

                html.Div([
                    dcc.Graph(id="bar-abs",   style={"width": f"{CHART_W}px", "height": f"{CHART_H}px"}),
                    dcc.Graph(id="bar-ratio", style={"width": f"{CHART_W}px", "height": f"{CHART_H}px"}),
                ], style={"display": "flex", "justifyContent": "space-between", "alignItems": "stretch"})
            ]
        )
    ]
)

# - Callback

@app.callback(
    Output("bar-abs", "figure"),
    Output("bar-ratio", "figure"),
    Input("n-slider", "value")
)
def update_charts(n):
    n = int(n)

    # A) Top-N by absolute vaccinations
    top_abs = (
        countries.sort_values("total_vaccinations", ascending=False)
                 .head(n)
                 .sort_values("total_vaccinations")
    )
    fig_abs = px.bar(
        top_abs, x="total_vaccinations", y="location", orientation="h",
        text="total_vaccinations",
        title=f"Top {n} countries — total vaccinations",
        labels={"total_vaccinations": "Total vaccinations", "location": ""},
        color_discrete_sequence=[palette_dash[0]],
    )
    fig_abs.update_traces(texttemplate="%{text:,.0f}", textposition="outside", cliponaxis=False)
    fig_abs.update_layout(
        width=CHART_W, height=CHART_H,
        paper_bgcolor="#ffffff", plot_bgcolor="#ffffff",
        margin=dict(l=10, r=10, t=40, b=10)
    )
    fig_abs.update_xaxes(showgrid=True, gridcolor="rgba(0,0,0,0.08)")

    # B) Top-N by vaccinations per capita
    top_ratio = (
        countries.sort_values("vaccinations_per_capita", ascending=False)
                 .head(n)
                 .sort_values("vaccinations_per_capita")
    )
    fig_ratio = px.bar(
        top_ratio, x="vaccinations_per_capita", y="location", orientation="h",
        text="vaccinations_per_capita",
        title=f"Top {n} countries — vaccinations / population",
        labels={"vaccinations_per_capita": "Vaccinations per capita", "location": ""},
        color_discrete_sequence=[palette_dash[0]],
    )
    fig_ratio.update_traces(texttemplate="%{text:.2%}", textposition="outside", cliponaxis=False)
    fig_ratio.update_layout(
        width=CHART_W, height=CHART_H,
        paper_bgcolor="#ffffff", plot_bgcolor="#ffffff",
        margin=dict(l=10, r=10, t=40, b=10)
    )
    fig_ratio.update_xaxes(tickformat=".0%", showgrid=True, gridcolor="rgba(0,0,0,0.08)")

    return fig_abs, fig_ratio

## Explanation
- **Slider (N)** → choose how many countries to display (5–20).  
- **Total vaccinations** → favors large-population countries (e.g., China, India, USA).  
- **Vaccinations per capita** → shows efficiency, often highlighting small states.  
- Values are forward-filled to use the latest totals for each country.

In [14]:
import socket
def get_free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = get_free_port()
app.run(debug=True, use_reloader=False, port=free_port)

![Top countries by vaccinations](../Assets/graph8_map_top20.png)

## Summary
Absolute and per-capita rankings tell different stories.  
Large countries dominate in total doses, while small nations often top per-capita charts.  
Viewing both side by side reveals global differences in rollout scale and efficiency.